In [1]:
import pandas as pd
df=pd.read_csv('train/train.csv')

In [2]:
df=df.dropna()

In [3]:
X=df.drop('label',axis=1)

In [4]:
y=df['label']

In [5]:
import nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [6]:
from nltk.stem import WordNetLemmatizer

In [7]:
messages=X.copy()
messages.reset_index(inplace=True)

In [8]:
lr = WordNetLemmatizer()
corpus = []
for i in range(0, len(messages)):
    print("Status: %s / %s" %(i, len(messages)), end="\r")
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [lr.lemmatize(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [9]:
corpus[0]

'house dem aide even see comey letter jason chaffetz tweeted'

In [10]:
from sklearn.model_selection import train_test_split
X_train_title, X_test_title, y_train_title, y_test_title = train_test_split(corpus, y, test_size = 0.2, random_state = 1)

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
max_vocab_title = 1500

# Create TF-IDF of title
tfidf_title = TfidfVectorizer(stop_words=stopwords.words('english'),ngram_range=(1,3), max_features=max_vocab_title)

sparse_tfidf_train_title = tfidf_title.fit_transform(X_train_title)

X_tfidf_train_title = sparse_tfidf_train_title.toarray()

In [12]:
import tensorflow as tf

In [13]:


model_title = tf.keras.Sequential()
model_title.add(tf.keras.layers.Dense(64,input_shape=(max_vocab_title,),activation='relu'))
model_title.add(tf.keras.layers.Dropout(0.2))
model_title.add(tf.keras.layers.Dense(16,activation='relu'))
model_title.add(tf.keras.layers.Dropout(0.2))
model_title.add(tf.keras.layers.Dense(1,activation='relu'))
model_title.add(tf.keras.layers.Activation('sigmoid'))
model_title.summary()
model_title.compile(loss='binary_crossentropy', optimizer='adam', metrics = ['accuracy', 'Recall'])

batch_size = 200
max_epochs = 10
early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)

model_title.fit(X_tfidf_train_title, y_train_title, batch_size = batch_size, epochs = max_epochs, callbacks=[early_stopping], validation_split = 0.2 , shuffle=True, verbose=1)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                96064     
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 16)                1040      
_________________________________________________________________
dropout_1 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 17        
_________________________________________________________________
activation (Activation)      (None, 1)                 0         
Total params: 97,121
Trainable params: 97,121
Non-trainable params: 0
____________________________________________________

In [14]:
sparse_tfidf_test_title = tfidf_title.transform(X_test_title)

X_tfidf_test_title = sparse_tfidf_test_title.toarray()

In [15]:
model_title_loss, model_title_accuracy, model_title_recall = model_title.evaluate(X_tfidf_test_title, y_test_title)

115/115 [==============================] - 0s 1ms/step - loss: 0.5198 - accuracy: 0.9163 - recall: 0.9627


In [16]:
model_title_loss, model_title_accuracy, model_title_recall

(0.51984041929245, 0.9163248538970947, 0.9626818299293518)

In [18]:
#model_title.save('rnn_title_tfidf.h5')

In [19]:
import pickle 
pickle.dump(tfidf_title, open('tranform.pkl', 'wb'))